# Watson OpenScale Mortgage Default Lab - Model Feed

This notebook should be run in a Watson Studio project with Python 3.6 or greater. This notebook will use the Watson Machine Learning client to send scoring requests to the mortgage default model. You can specify the number of requests sent in the cells below.

Replace the credentials below with those of your Watson Machine Learning instance with the mortgage default model deployed.

In [ ]:
WML_CREDENTIALS = {
    "apikey": "key",
    "iam_apikey_description": "description",
    "iam_apikey_name": "auto-generated-apikey",
    "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
    "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::",
    "instance_id": "instance_id",
    "url": "https://us-south.ml.cloud.ibm.com",
}

You can generate a Cloud API key [here](https://cloud.ibm.com/iam/apikeys). The API key is used to connect to OpenScale to send feedback data.

In [ ]:
CLOUD_API_KEY = "xxxxxxxxxxxxxxxxx"

In the cell below, ensure that the model and deployment names are the same used for creating the model. Set the minimum and maximum number of records you wish to send to the model for scoring, and set the number of feedback records to send.

In [ ]:
MODEL_NAME = 'Mortgage Default'
DEPLOYMENT_NAME = 'Mortgage Default - Production'

MIN_PAYLOADS = 101
MAX_PAYLOADS = 125
FEEDBACK_RECORDS = 50

Using the credentials from above, create a Watson Machine Learning API client.

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

Get the deployment and model IDs from the WML client.

In [ ]:
mortgage_deployment_id = None
model_deployment_ids = client.deployments.get_uids()
for deployment_id in model_deployment_ids:
    deployment = client.deployments.get_details(deployment_id)
    model_id = deployment['entity']['deployable_asset']['guid']
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        print('Deployment id:', deployment_id)
        mortgage_deployment_id = deployment_id
        print('Model id:', model_id)
        break

Get the scoring URL of the model.

In [ ]:
deployment_details = client.deployments.get_details(mortgage_deployment_id)
scoring_url = deployment_details['entity']['scoring_url']
scoring_url

Get the training data for the model and feedback feeds, dropping the transaction ID.

In [ ]:
import pandas as pd

url = 'https://raw.githubusercontent.com/emartensibm/openscale-demos/master/mortgage-default/data/Mortgage_Full_Records.csv'
df_raw = pd.read_csv(url)
df = df_raw.drop('ID', axis=1)
df.head()

features = list(df)
payload_set = df.values.tolist()

Set up the data structure for scoring requests. The ID field was not a feature used to train the model, but is still used to identify the transactions. For this reason, we will send it as metadata with the request, and it will be stored in the OpenScale payload log.

Additionally, we will omit the last column from our training data, since it includes the label (MortgageDefault), which will not be used in the scoring.

In [ ]:
scoring_payload = {
    "fields": features[:-1],
    "values": [],
    "meta":{
        "fields": ["ID"],
        "values": []
    }
}

Randomly select fields from the data to score, and randomly assign an integer ID to each. When we append the payload, omit the final column from the data, which is the label data.

In [ ]:
import random
import string

letters = string.digits

for _ in range(0, random.randint(MIN_PAYLOADS, MAX_PAYLOADS)):
    value_to_score = random.choice(payload_set)
    scoring_payload['values'].append(value_to_score[:-1])
    scoring_payload['meta']['values'].append([int(''.join(random.choices(letters, k=8)))])

In [ ]:
scoring_payload

Send the scoring request to the model.

In [ ]:
predictions = client.deployments.score(scoring_url, scoring_payload)

Print out the returned predictions.

In [ ]:
predictions

## Feedback data

This section of the notebook sends scored feedback data to OpenScale for the accuracy monitor. First, we need to install the OpenScale Python client.

In [ ]:
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1

Next, we'll use the API key supplied above to get the Watson OpenScale GUID.

In [ ]:
from ibm_ai_openscale import APIClient
import requests
from ibm_ai_openscale.utils import get_instance_guid

WOS_GUID = get_instance_guid(api_key=CLOUD_API_KEY)
WOS_CREDENTIALS = {
    "instance_guid": WOS_GUID,
    "apikey": CLOUD_API_KEY,
    "url": "https://api.aiopenscale.cloud.ibm.com"
}

if WOS_GUID is None:
    print('Watson OpenScale GUID NOT FOUND')
else:
    print(WOS_GUID)

Using the Watson OpenScale GUID, we'll open a client connection to OpenScale.

In [ ]:
ai_client = APIClient(aios_credentials=WOS_CREDENTIALS)
ai_client.version

Get the OpenScale subscription for the model, using the model name specified above.

In [ ]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
for sub in subscriptions_uids:
    if ai_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == MODEL_NAME:
        subscription = ai_client.data_mart.subscriptions.get(sub)

In [ ]:
subscription.get_details()

Randomly select a set number of records to be used as feedback data.

In [ ]:
feedback_payload = []
for _ in range(0, FEEDBACK_RECORDS):
    feedback_payload.append(random.choice(payload_set))
print(feedback_payload[0])

Store the feedback data in OpenScale.

In [ ]:
subscription.feedback_logging.store(feedback_payload)

# Congratulations!

If all cells have run successfully, you have submitted prediction and feedback data to your model. You can save a version of this notebook using the **File** menu above, and schedule it to run periodically to continue sending data for OpenScale to monitor.

This menu can be scheduled to run regularly to feed data to the model and trigger the OpenScale monitors. To schedule it, click **File**->**Save Version**. Once the notebook has saved, click the **clock icon** to the right of the **share icon** in the top center of the screen and follow the prompts. Running it hourly will provide the best results.